# Run a training script as a command job

You can use the Python SDK for Azure Machine Learning to submit scripts as command jobs. By using jobs, you can easily keep track of the input parameters and outputs when training a machine learning model.

## Before you start

You'll need the latest version of the  **azureml-ai-ml** package to run the code in this notebook. Run the cell below to verify that it is installed.

> **Note**:
> If the **azure-ai-ml** package is not installed, run `pip install azure-ai-ml` to install it.

In [1]:
pip show azure-ai-ml

Name: azure-ai-ml
Version: 1.16.1
Summary: Microsoft Azure Machine Learning Client Library for Python
Home-page: https://github.com/Azure/azure-sdk-for-python
Author: Microsoft Corporation
Author-email: azuresdkengsysadmins@microsoft.com
License: MIT License
Location: /anaconda/envs/azureml_py38/lib/python3.8/site-packages
Requires: tqdm, marshmallow, azure-core, azure-storage-file-share, isodate, msrest, pyjwt, azure-storage-file-datalake, pyyaml, typing-extensions, pydash, azure-mgmt-core, opencensus-ext-logging, strictyaml, azure-common, opencensus-ext-azure, colorama, azure-storage-blob, jsonschema
Required-by: 
Note: you may need to restart the kernel to use updated packages.


## Connect to your workspace

With the required SDK packages installed, now you're ready to connect to your workspace.

To connect to a workspace, we need identifier parameters - a subscription ID, resource group name, and workspace name. Since you're working with a compute instance, managed by Azure Machine Learning, you can use the default values to connect to the workspace.

In [2]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient

try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

In [3]:
# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

Found the config file in: /config.json


## Initiate a command job

Run the cell below to train a classification model to predict diabetes. The model is trained by running the **train-model-parameters.py** script that can be found in the **src** folder. It uses the **diabetes.csv** file as the training data. 

- `code`: specifies the folder that includes the script to run.
- `command`: specifies what to run exactly.
- `environment`: specifies the necessary packages to be installed on the compute before running the command.
- `compute`: specifies the compute to use to run the command.
- `display_name`: the name of the individual job.
- `experiment_name`: the name of the experiment the job belongs to.

Note that the command used to test the script in the terminal is the same as the command in the configuration of the job below. 

In [4]:
from azure.ai.ml import command

# configure job

job = command(
    code="./src",
    command="python train-model-parameters.py --training_data diabetes.csv",
    environment="AzureML-sklearn-0.24-ubuntu18.04-py37-cpu@latest",
    compute="aml-cluster",
    display_name="diabetes-train-script",
    experiment_name="diabetes-training"
    )

# submit job
returned_job = ml_client.create_or_update(job)
aml_url = returned_job.studio_url
print("Monitor your job at", aml_url)

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading src (0.56 MBs): 100%|███████

Monitor your job at https://ml.azure.com/runs/clever_head_gsds3xg2mr?wsid=/subscriptions/3717cba1-f6f1-4e4a-9d28-1133c9b08740/resourcegroups/rg-dp100-l0b4978440a624bb888/workspaces/mlw-dp100-labs&tid=f94bf4d9-8097-4794-adf6-a5466ca28563
